In [33]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import geopandas as gpd

In [34]:
def distance_between_two_points(lat1, lat2, lon1, lon2):

     """
     A procedure to get the distance between two points with latitdues and longitudes
     INPUTS:
        lat1: -> latitude of the first point in degree 
        lat2: -> latitude of the second point in degree 
        lon1: -> longitude of the first point in degree 
        lon2: -> longitude of the second point in degree 
     """    
    # The numpy module contains a function named
    # radians which converts from degrees to radians.

     #lon1a = np.radians(lon1)
    # lon2a = np.radians(lon2)
     lat1a = np.radians(lat1)
     lat2a = np.radians(lat2)
      
    # Haversine formula
    
     dlon = np.radians(lon2 - lon1)
     dlat = np.radians(lat2 - lat1)
     a = np.sin(dlat / 2)**2 + np.cos(lat1a) * np.cos(lat2a) * np.sin(dlon / 2)**2
 
    
     c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
     r = 6371
      
     # calculate the result

     return np.round(c * r, 6)

In [35]:
# Create an engine to connect to sqlite database
engine = create_engine('sqlite:///water_points.db')
conn = engine.connect()
df = pd.read_sql_table('etlTable', con=conn)

In [36]:
osun = df[df.state=='Osun']
osun.describe()

,id,resident_latitude,latitude,resident_longitude,longitude,distance_in_km,population
count,5126.000000,5126.000000,5126.000000,5126.000000,5126.000000,5126.000000,5126.000000
mean,42207.405774,7.653438,7.650871,4.562282,4.557402,0.648490,2166.447718
std,5796.233700,0.238095,0.238076,0.217603,0.217546,0.273467,383.542700
min,22809.000000,7.062604,7.060309,4.036361,4.032004,0.005814,1500.000000
25%,39801.500000,7.488223,7.485158,4.375920,4.372685,0.442524,1838.250000
50%,41339.500000,7.661360,7.658886,4.570636,4.565806,0.639785,2165.000000
75%,42685.750000,7.847734,7.845455,4.733466,4.728331,0.871640,2503.000000
max,72345.000000,8.063658,8.061898,5.062381,5.055003,1.209815,2829.000000


In [37]:
lat1 = input("Enter your latitude in the range 7.060309 - 8.061898 and press enter:    ")
long1 = input("Enter your longitude in the range 4.032004 - 5.055003 and press enter:   ")
lat1 = float(lat1)
long1 = float(long1)

Enter your latitude in the range 7.060309 - 8.061898 and press enter:    7.67891
Enter your longitude in the range 4.032004 - 5.055003 and press enter:   4.98231


In [38]:
sort_osun = osun.sort_values(by="distance_in_km")
sort_osun.head()

,id,resident_latitude,latitude,resident_longitude,longitude,status_id,water_source,water_technology,state,lga,...,status,subjective_quality,location,photo_lnk,lat_lon_deg,New Georeferenced Column,management_clean,water_source_category,distance_in_km,population
37815,37816,7.936749,7.936700,4.719528,4.719508,Yes,Borehole,Hand Pump,Osun,Ifelodun,...,Functional and in use but in bad shape,Acceptable quality,BHS Area 11,https://akvoflow-55.s3.amazonaws.com/images/74...,"(7.9367°, 4.7195083°)",POINT (4.7195083 7.9367),Community Management,Well,0.005814,1848
42699,42700,7.367612,7.367552,4.199351,4.199305,Yes,Protected Shallow Well,Hand Pump,Osun,Irewole,...,Functional and in use but in bad shape,No because of Taste,FATIMA,https://akvoflow-55.s3.amazonaws.com/images/d7...,"(7.3675517°, 4.199305°)",POINT (4.199305 7.3675517),Community Management,Well,0.008385,2484
44869,44870,7.952230,7.952182,4.989929,4.989852,Yes,Borehole,Hand Pump,Osun,Ifedayo,...,Functional (and in use),Acceptable quality,ejemu,https://akvoflow-55.s3.amazonaws.com/images/61...,"(7.9521817°, 4.9898517°)",POINT (4.9898517 7.9521817),Private Operator/Delegated Management,Well,0.010027,1779
42175,42176,7.642614,7.642500,4.189247,4.189097,Yes,Protected Shallow Well,Hand Pump,Osun,Iwo,...,Functional (and in use),No because of Colour,united Middle Sch Araromi,https://akvoflow-55.s3.amazonaws.com/images/72...,"(7.6425°, 4.1890967°)",POINT (4.1890967 7.6425),Community Management,Well,0.020871,2796
39127,39128,7.526364,7.526254,4.454156,4.453982,No,Borehole,Mechanized Pump,Osun,Ife North,...,Non-functional Technical breakdown,Acceptable quality,Oke osin,https://akvoflow-55.s3.amazonaws.com/images/f8...,"(7.5262542°, 4.4539824°)",POINT (4.4539824 7.5262542),Community Management,Well,0.022759,1668


In [39]:
def closest_taps(df1, num_taps=10):
    
     #Create a column for distance between a household and nearest water points
    df = df1.copy()
    distance = []
    for row in df.itertuples(index=False):
        distance.append(distance_between_two_points(lat1, row.latitude, lon1, row.longitude) )
    
    df['dist'] = distance
    df = df[(df.subjective_quality=="Acceptable quality") & (df.status=="Functional (and in use)") & (df.water_source=="Borehole")]
    closest_taps = df.sort_values(by="dist")
    # Get the top points
    closest_taps = closest_taps[['location', 'population', 'dist']][:num_taps]
    
    return closest_taps
    

In [40]:
taps = closest_taps(osun)
taps

,location,population,dist
41205,Akoda,2727,1.234588
44719,Akoda,2328,1.261788
46239,Oloki,2284,1.679460
42002,Akoda Garage,1513,1.726092
42411,Ededimeji,1502,1.972001
40360,Ogobi Oja,1509,2.313698
39814,Olorubu Ologobi,1552,2.516059
37893,"DIG,s Quaters",1693,2.874497
40735,Obanisola,2564,2.983872
39125,Olohunda Odomu,1811,3.276555
